In [2]:
import os
import json
from pathlib import Path

In [3]:
nltk_data_path = Path("~/.cache/nltk_data").expanduser()
nltk_data_path.mkdir(parents=True, exist_ok=True)
os.environ["NLTK_DATA"] = str(nltk_data_path)

In [4]:
# Deps for PDF parsing
!pip install "unstructured[pdf]"
!sudo apt-get install -y poppler-utils tesseract-ocr

# I can't even remember why we need this one
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [5]:
!pip install text_generation==0.6.1
!pip install langchain==0.0.354
!pip install langchain-community==0.0.8
!pip install langchain-core==0.1.5
!pip install langsmith==0.0.77

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install

In [6]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [7]:
!pip uninstall faiss faiss-gpu -y
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [8]:
import torch
import transformers
from langchain.schema import AIMessage, SystemMessage, HumanMessage

In [9]:
%load_ext autoreload
%autoreload 2
from util import (
    load_tokenizer,
    load_tgi_chat_model,
    HuggingFaceChatModel,
    VectorStoreRetrieverWithTextSplitter,
    parse_pdf,
    HuggingFaceChatModelWithBatchSupport
)

In [10]:
print(f"{transformers.__version__=}")
print(f"{torch.__version__=}")
print(f"{torch.version.cuda=}")
print(f"{torch.cuda.is_available()=}")
print(f"{torch.cuda.device_count()=}")

transformers.__version__='4.36.0'
torch.__version__='2.1.1+cu118'
torch.version.cuda='11.8'
torch.cuda.is_available()=True
torch.cuda.device_count()=1


In [11]:
!nvidia-smi

Tue Feb  6 20:15:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   52C    P8              22W / 420W |  19590MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
!curl http://tgi:80/info

{"model_id":"TheBloke/OpenHermes-2.5-Mistral-7B-AWQ","model_sha":"a0e39e3a0ca6767307293819acb7c38e7a18cd31","model_dtype":"torch.float16","model_device_type":"cuda","model_pipeline_tag":"text-generation","max_concurrent_requests":128,"max_best_of":2,"max_stop_sequences":4,"max_input_length":4096,"max_total_tokens":6096,"waiting_served_ratio":1.2,"max_batch_total_tokens":86160,"max_waiting_tokens":20,"validation_workers":2,"version":"1.3.4","sha":"630800eed37b15c4b0c9eb8e6ab47212026720f7","docker_label":"sha-630800e"}

In [13]:
model_name='teknium/OpenHermes-2.5-Mistral-7B'
tgi_url = "http://tgi:80"

In [27]:
tokenizer = load_tokenizer(model_name)
chat_model = load_tgi_chat_model(
    tokenizer=tokenizer,
    inference_server_url=tgi_url,
    max_new_tokens=512,
    timeout=900,
    do_sample=False,
    temperature=0.4,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
example = """\
Context:
----
Utility: These cards do something beneficial. For example, Call in a Favor lets you take resources of your choice from the supply.

Mandatory Quest: You can force an opponent to undertake a minor but pressing task before finishing other Quests. See “Mandatory Quests” on page 11 for more information.

2. Reassign Agent: After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space. In effect, that player takes an additional turn with that Agent. Players reassign Agents in order, starting with the action space numbered “1.” You cannot reassign an Agent to Waterdeep Harbor.

Some game effects also allow players to reassign Agents during play.
----

Question: When can I assign agents?

Quotes:
----
After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space.
----
----
Some game effects also allow players to reassign Agents during play.
----
"""

In [16]:
query = """\
Given a context and a question, quote the most relevant parts of the context that relate to the question. Do not attempt to answer the question, simply quote the relevant parts of the context. The following format will be used:

Context:
----
<document text here>
----

Question: <question text here>

Quotes:
----
<quote 1 here>
----
----
<quote 2 here>
----
----
<quote 3 here>
----

Now the actual context and question:

Context:
----
"ONE-SHOT” TREASURES
A Treasure card that says “Usable once only” is often called a “one-shot” Treasure. Most of these are used during combat to strengthen the munchkins or the monsters, and may be played from your hand or from the table. Some have other effects, however, so read the card carefully! Discard these cards as soon as the combat is over or their effect is resolved.
One-shot Items with a Gold Piece value may be sold for levels, just like other Items.

OTHER TREASURES
Other Treasure cards (like Go Up a Level cards) are not Items. Most of these cards say when they can be played, and whether they stay in play or are discarded. A couple of specific examples: Go Upa Level cards may be played on yourself or any other player at any time, even during combat. Discard them once they are played. Exception: You cannot play a Go Up a Level card to give a player the winning level!
Hireling may be played at any time, on any turn. You cannot give a Hireling an Item to carry while you are in combat, however.
----

Question: Can I play a Go Up a Level card during combat?

Quotes: \
"""

conversation = [
    SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

chat_model.invoke(conversation)

AIMessage(content='"One-shot Items with a Gold Piece value may be sold for levels, just like other Items."\n\n"Go Upa Level cards may be played on yourself or any other player at any time, even during combat. Discard them once they are played. Exception: You cannot play a Go Up a Level card to give a player the winning level!"')

In [17]:
query = """\
Given a context and a question, quote the most relevant parts of the context that relate to the question. Do not attempt to answer the question, simply quote the relevant parts of the context verbatim. The following format will be used:

Question: <question text here>

Context:
----
<document text here>
----

Quotes:
----
<quote 1 here>
----
----
<quote 2 here>
----
----
<quote 3 here>
----

Example:

Question: When can I assign agents?

Context:
----
Utility: These cards do something beneficial. For example, Call in a Favor lets you take resources of your choice from the supply.

Mandatory Quest: You can force an opponent to undertake a minor but pressing task before finishing other Quests. See “Mandatory Quests” on page 11 for more information.

2. Reassign Agent: After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space. In effect, that player takes an additional turn with that Agent. Players reassign Agents in order, starting with the action space numbered “1.” You cannot reassign an Agent to Waterdeep Harbor.

Some game effects also allow players to reassign Agents during play.
----

Quotes:
----
After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space.
----
----
Some game effects also allow players to reassign Agents during play.
----

Now the actual context and question you are to answer:

Question: Can I play a Go Up a Level during combat?

Context:
----
When You May Take Actions

You may perform these actions at any time:
- Discard a Class or Race.
- Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: 
- Trade an Item with another player (the other player may not be in combat, either).
- Change which Items you have equipped.
- Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn:
- Play a new Class or Race card (at any time).
- Sell Items for levels (except when you are in combat).
- Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).
----

Quotes:
----
"""

conversation = [
    SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

print(chat_model.invoke(conversation).content)

You may perform these actions at any time, as long as you are not in combat:
- Trade an Item with another player (the other player may not be in combat, either).
- Change which Items you have equipped.
- Play a card that you have just received (some cards may be played even during combat; see above).

You may perform these actions on your own turn:
- Play a new Class or Race card (at any time).
- Sell Items for levels (except when you are in combat).
- Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).


In [18]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. Do not prefix extracted parts with "-" or "*". 

Remember, *DO NOT* edit the extracted parts of the context.

> Question: {question}
> Context:
>>>
{context}
>>>
Extracted relevant parts:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"],
)

question = "Can I play a Curse while in combat?"

context = """\
When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3)."""

context2 = """\
COMBAT
To fight a monster, compare its combat strength to yours. Combat strength is the total of Level plus all modifiers - positive or negative - given by Items and other cards. If the monster's combat strength is equal to yours, or greater, you lose the combat and must Run Away (see p. 5). If your combat strength totals more than the monster's — note that monsters win ties! — you kill it and goupa level (two levels for some big monsters). You'll also get the number of Treasures shown on its card.
Sometimes a card will let you get rid of the monster without killing it. This is still “winning,” but you don't get a level. Unless the ability says otherwise, you don’t get the Treasures, either. If the last monster is removed from a combat, it ends instantly.
Some monster cards have special powers that affect combat — a bonus against a Race or Class, for instance. Be sure to check these!
You and the other players may play one-shot Treasures or use Class or Race abilities to help or harm you in your combat. Some Door cards may also be played into a combat, such as monster enhancers (see below).
If you kill a monster (or monsters!), discard the monster(s) and any other cards played, and claim your rewards. But note: someone may play a hostile card on you, or use a special power, just as you think you have won. When you kill a monster, you must wait a reasonable time, defined as about 2.6 seconds, for anyone else to speak up. After that, you have really killed the monster, and you really get the level(s) and Treasures, though they can still whine and argue.
"""

conversation = [
    # SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

result = (prompt | chat_model).invoke({"question": question, "context": context})

print(context)
print()
print()
print("Extracted:")
print(result.content)

# print(chat_model.invoke(conversation).content)

When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).


Extracted:
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see abo

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
prompt_template = """Clearly restate the following excerpt from the board game Munchkin, without changing the meaning or the information conveyed. The rewritten text should be in the same language as the original text.
> Context:
>>>
{context}
>>>
Rewritten text:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context"],
)

question = "Can I play a Curse while in combat?"

context = """\
When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3)."""

context2 = """\
COMBAT
To fight a monster, compare its combat strength to yours. Combat strength is the total of Level plus all modifiers - positive or negative - given by Items and other cards. If the monster's combat strength is equal to yours, or greater, you lose the combat and must Run Away (see p. 5). If your combat strength totals more than the monster's — note that monsters win ties! — you kill it and goupa level (two levels for some big monsters). You'll also get the number of Treasures shown on its card.
Sometimes a card will let you get rid of the monster without killing it. This is still “winning,” but you don't get a level. Unless the ability says otherwise, you don’t get the Treasures, either. If the last monster is removed from a combat, it ends instantly.
Some monster cards have special powers that affect combat — a bonus against a Race or Class, for instance. Be sure to check these!
You and the other players may play one-shot Treasures or use Class or Race abilities to help or harm you in your combat. Some Door cards may also be played into a combat, such as monster enhancers (see below).
If you kill a monster (or monsters!), discard the monster(s) and any other cards played, and claim your rewards. But note: someone may play a hostile card on you, or use a special power, just as you think you have won. When you kill a monster, you must wait a reasonable time, defined as about 2.6 seconds, for anyone else to speak up. After that, you have really killed the monster, and you really get the level(s) and Treasures, though they can still whine and argue.
"""

conversation = [
    # SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

result = (prompt | chat_model).invoke({"context": context})

print(context)
print()
print()
print("Rwritten:")
print(result.content)

# print(chat_model.invoke(conversation).content)

When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).


Rwritten:
At what times you may take actions: 
You are allowed to perform these actions at any time: Discard a Class or Race, Play a Go Up a Level or Hireling, and Play a Curse. 
You may perform these actions at any time, provided you are not in combat: Trade an Item with another player (the other playe

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """Answer the following question based on the given context. Provide your reasoning first followed by the answer.

> Context:
>>>
{context}
>>>
> Question: {question}
> Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"],
)

question = "Can I play a Curse while in combat?"

context = """\
When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3)."""

context2 = """\
COMBAT
To fight a monster, compare its combat strength to yours. Combat strength is the total of Level plus all modifiers - positive or negative - given by Items and other cards. If the monster's combat strength is equal to yours, or greater, you lose the combat and must Run Away (see p. 5). If your combat strength totals more than the monster's — note that monsters win ties! — you kill it and goupa level (two levels for some big monsters). You'll also get the number of Treasures shown on its card.
Sometimes a card will let you get rid of the monster without killing it. This is still “winning,” but you don't get a level. Unless the ability says otherwise, you don’t get the Treasures, either. If the last monster is removed from a combat, it ends instantly.
Some monster cards have special powers that affect combat — a bonus against a Race or Class, for instance. Be sure to check these!
You and the other players may play one-shot Treasures or use Class or Race abilities to help or harm you in your combat. Some Door cards may also be played into a combat, such as monster enhancers (see below).
If you kill a monster (or monsters!), discard the monster(s) and any other cards played, and claim your rewards. But note: someone may play a hostile card on you, or use a special power, just as you think you have won. When you kill a monster, you must wait a reasonable time, defined as about 2.6 seconds, for anyone else to speak up. After that, you have really killed the monster, and you really get the level(s) and Treasures, though they can still whine and argue.
"""

rewritten_context = """\
At any time, you may perform the following actions: discard a class or race, play a go up a level or hireling, or play a curse. Additionally, when you are not in combat, you may also trade an item with another player (who must also not be in combat), change your equipped items, or play a card that you have just received (some cards can be played during combat, see above). On your turn, you may also perform the following actions: play a new class or race card, sell items for levels (except when in combat), and play an item (most items cannot be played during combat, but some one-shot items can; see page 3).
"""

rewrite_prompt_template = """Reword and improve clarity of the following excerpt from the board game Munchkin, without changing the meaning or the information conveyed. Use Markdown to format the text.

> Context:
>>>
{context}
>>>
Rewritten text:"""

rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["context"],
)

context_to_use = (rewrite_prompt | chat_model).invoke({"context": context}).content
question = "Can I play a Go Up a Level card while in combat?"
result = (prompt | chat_model).invoke({"question": question, "context": context_to_use})

print(context_to_use)
print()
print()
print("Answer:")
print(result.content)

# print(chat_model.invoke(conversation).content)

**When You May Take Actions**

You have the following options at any time, regardless of whether you are in combat:

- Discard a Class or Race.
- Play a Go Up a Level or Hireling.
- Play a Curse.

You also have these options as long as you are not in combat:

- Trade an Item with another player (the other player must also not be in combat).
- Change which Items you have equipped.
- Play a card that you have just received (some cards can be played during combat; see above).

During your own turn, you can perform the following actions:

- Play a new Class or Race card (at any time).
- Sell Items for levels (except when you are in combat).
- Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).


Answer:
First, we need to identify the options available during combat. According to the context, the available options during combat are:

- Discard a Class or Race.
- Play a Go Up a Level or Hireling.
- Play a Curse.

Since playing a Go Up a Level card

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """Answer the following question based on the given context. Provide your reasoning first followed by the answer.

> Context:
>>>
{context}
>>>
> Question: {question}
> Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"],
)

context = """\
Items are the most common Treasures in Munchkin. They have a Gold Piece value, and a "No Value" card is equivalent to zero Gold Pieces. Items that are "carried" are considered to be in your possession, while those that are providing you with a bonus are "equipped." To indicate that an Item is not equipped, turn the card sideways. You cannot alter the status of your Items during combat or while running away.

Any player can carry any Item (except for extra Big items; see below), but you may equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) . . . unless you have a card that lets you ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise. If you are carrying two Headgear cards, for instance, you can equip only one of them at a time.

Some Items have restrictions, such as the Mace of Sharpness, which can only be wielded by a Cleric. Its bonus only counts for someone who is, at the moment, a Cleric.

You cannot discard Item cards "just because." You may sell Items for a level, trade Items with other players, or give an Item to another player who wants it (see below). You may discard Items to power certain Class and Race abilities. And a Curse or a monster's Bad Stuff (see p. 5) may force you to get rid of something!

Big Items: You may carry any number of Small items, but only one Big one. (Any item not marked Big is considered Small.) You may not discard one Big item to play another; you must sell the first Item, trade it, lose it to a Curse or Bad Stuff, or discard it to power a Class or Race ability."""


rewrite_prompt_template = """Reword and improve clarity of the following excerpt from the board game Munchkin, without changing the meaning or the information conveyed. Use Markdown to format the text.

> Context:
>>>
{context}
>>>
Rewritten text:"""

rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["context"],
)

context_to_use = (rewrite_prompt | chat_model).invoke({"context": context}).content
question = "Can I carry multiple big items so long as only one is equipped?"
result = (prompt | chat_model).invoke({"question": question, "context": context_to_use})

print(context_to_use)
print()
print()
print("Answer:")
print(result.content)

# print(chat_model.invoke(conversation).content)

Items are the most common treasures in Munchkin, possessing a gold piece value and a "No Value" card equivalent to zero gold pieces. Items that are "carried" are considered to be in your possession, while those that provide a bonus are "equipped." To indicate that an item is not equipped, turn the card sideways. You cannot alter the status of your items during combat or while running away.

Any player can carry any item (except for extra big items; see below), but you may equip only one headgear, one suit of armor, one pair of footgear, and two "I Hand" items (or one "2 Hands" item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards states otherwise. If you are carrying two headgear cards, for instance, you can equip only one at a time.

Some items have restrictions, such as the Mace of Sharpness, which can only be wielded by a Cleric. Its bonus only counts for someone who is, at the moment, a Cleric.

You cannot discar

In [34]:
why_wrong_prompt_template = """The following is a context, question, and incorrect answer. It is absolutely certain the answer is incorrect. Your job is to figure out why the answer is incorrect based on the context.

> Context:
>>>
{context}
>>>
> Question: {question}
> Answer: {answer}
> Explanation: Assuming the answer is wrong, why is it wrong?"""

why_wrong_prompt = PromptTemplate(
    template=why_wrong_prompt_template,
    input_variables=["question", "context", "answer"],
)

print((why_wrong_prompt | chat_model).invoke({
    "question": question,
    "answer": result.content,
    "context": context_to_use,
}).content)

The answer is incorrect because it misunderstands the difference between carrying and equipping items. While it is true that a player can carry multiple big items, only one can be equipped at a time. The answer conflates the two concepts, leading to an incorrect understanding of the rules.
